In [2]:
import openai
import datasets
from transformers import AutoTokenizer, DataCollatorForSeq2Seq,\
    AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, HfArgumentParser
# from helpers import prepare_dataset_code_summary, compute_rouge_and_bleu
from tqdm import tqdm
import os
import json

In [3]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [4]:
from transformers import SummarizationPipeline, pipeline
from torch import Tensor

In [5]:
dataset = datasets.load_dataset('json', data_files="./codesearchnet-corpus/python/final/jsonl/train/python_train_0.jsonl")
dataset = dataset['train']

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 452.90it/s]


Dataset json downloaded and prepared to /Users/ryanchhong/.cache/huggingface/datasets/json/default-3f5d1d0082396c1a/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 533.63it/s]


In [64]:
from transformers import SummarizationPipeline
from torch import Tensor


class CustomSummaryGenerationPipeline(SummarizationPipeline):

    def preprocess(self, inputs):
        max_seq_length = self.tokenizer.model_max_length
        
        examples_with_prompts = list(map(lambda x: "Summarize Python: " + x, inputs))
        # print(examples_with_prompts)
        tokenized_examples = self.tokenizer(
            examples_with_prompts,
            truncation=True,
            max_length=max_seq_length,
            padding='max_length',
            return_tensors = 'pt'
        )
        return tokenized_examples

    def _forward(self, model_inputs):
        outputs = self.model.generate(**model_inputs)
        return {"encoded_pred": outputs}

    def postprocess(self, model_outputs):
        # print(model_outputs)
        encoded_predictions = model_outputs["encoded_pred"]
        decoded_predictions = self.tokenizer.batch_decode(encoded_predictions, skip_special_tokens=True)
        # decoded_predictions = [pred.strip() for pred in decoded_predictions]
        return decoded_predictions

In [65]:
my_pipeline = pipeline(
    task = "summarization",
    model = "./single_line_summarization_model/",
    pipeline_class = CustomSummaryGenerationPipeline
)

In [66]:
store = []
for out in tqdm(my_pipeline(KeyDataset(dataset, 'code')[:2])):
    store.append(out)

100%|██████████| 2/2 [00:00<00:00, 76959.71it/s]


In [68]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
test = [
["get the metrics of a model in a web browser", "check if a sandbox is running", "convert a string `result` to a Python object"],
["create a dictionary `params_dict` with key `key` and timeoutSecs `10", "check if parameters in dictionary `params_dict` are updated with dictionary `kwargs`", "get json data from a key `key` in seconds", "send a json request to the API '3/Models.json' with timeout `timeout", "dump json object `result` to console", "check if a sandbox is running", "convert a string `result` to a Python object"],
["delete a model `key` from a database", "delete a model `key` from the server", "check if key 'f00b4r' exists in result", "delete a model `result` from a python python script", "convert a string `result` to a Python object"]
]

In [9]:

input = ["Summarize in a couple of sentences:\n\n" + "\n".join(line_summary) for line_summary in test]
gpt_response = openai.Completion.create(
    model="text-davinci-003",
    prompt=input,
    temperature=0.7,
    max_tokens=64,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)


In [10]:
print(gpt_response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis article describes how to use the Werkzeug library in Python to get the metrics of a model in a web browser, check if a sandbox is running, and convert a string to a Python object."
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "text": " `response`\n\nThis code creates a dictionary with a key and timeoutSecs, then updates the parameters with a kwargs dictionary. It then sends a json request to the API and dumps the response to console. It also checks if a sandbox is running, and converts the response string to a Python object."
    },
    {
      "finish_reason": "length",
      "index": 2,
      "logprobs": null,
      "text": "\n\nTo delete a model key from a database or server, you can use the delete() method. To check if a key exists in a result, use the in operator. To delete a model result from a Python script, use 

In [26]:
def nth_fibonacci(n):
    if(n < 0):
        raise Exception('Invalid Input')
    
    if(n < 2):
        return 1

    curr = 1
    prev = curr

    for _ in range(n):
        prev, curr= curr, curr + prev
    
    return curr

In [29]:
nth_fibonacci(1)

1